In [49]:
import numpy as np
import matplotlib as plt
from sklearn.mixture import GaussianMixture as GMM
import pandas as pd
from lenstools.statistics.ensemble import Series,Ensemble
from lenstools.statistics.constraints import Emulator
from lenstools.statistics.contours import ContourPlot
from lenstools.statistics.samplers import emcee_sampler
from itertools import cycle
import seaborn as sns
import emcee
from multiprocessing import Pool

In [11]:
datapath = "../power_spectrum.csv"
data = pd.read_csv(datapath)
data.pop("Unnamed: 0")
ell_bins = [f"ell{i}" for i in range(37)]
ell = np.logspace(np.log10(500), np.log10(5000))
params = ["H", "OMEGA_M", "Omega_L", "W0", "WA", "Z", "ANGLE"]
data.head()

,H,OMEGA_M,OMEGA_L,W0,WA,Z,ANGLE,ell0,ell1,ell2,...,ell27,ell28,ell29,ell30,ell31,ell32,ell33,ell34,ell35,ell36
0,0.72,0.3,0.7,-1.0,0.0,1.0,3.5,2.147647e-09,2.308267e-09,2.235948e-09,...,2.309436e-10,1.998192e-10,2.201959e-10,1.615317e-10,1.359018e-10,1.517825e-10,1.221337e-10,1.034097e-10,8.253211e-11,8.406824e-11
1,0.72,0.3,0.7,-1.0,0.0,1.0,3.5,3.152693e-09,1.507890e-09,1.634781e-09,...,2.448453e-10,2.317264e-10,1.674993e-10,1.644215e-10,1.584156e-10,1.398838e-10,1.137945e-10,9.118034e-11,8.772158e-11,7.803870e-11
2,0.72,0.3,0.7,-1.0,0.0,1.0,3.5,1.860668e-09,1.525577e-09,1.554946e-09,...,2.194509e-10,1.845656e-10,1.697013e-10,1.554389e-10,1.276890e-10,1.302592e-10,1.154453e-10,8.228746e-11,8.670791e-11,6.841955e-11
3,0.72,0.3,0.7,-1.0,0.0,1.0,3.5,2.233512e-09,2.598031e-09,7.218929e-10,...,2.379459e-10,2.217191e-10,1.930525e-10,1.489930e-10,1.366053e-10,1.354503e-10,1.018555e-10,9.417659e-11,8.339234e-11,7.193055e-11
4,0.72,0.3,0.7,-1.0,0.0,1.0,3.5,2.479310e-09,1.973829e-09,1.812698e-09,...,2.149722e-10,2.055646e-10,2.206855e-10,1.813513e-10,1.597146e-10,1.139527e-10,1.177881e-10,9.937584e-11,9.550918e-11,8.252748e-11


In [12]:
data.describe()

,H,OMEGA_M,OMEGA_L,W0,WA,Z,ANGLE,ell0,ell1,ell2,...,ell27,ell28,ell29,ell30,ell31,ell32,ell33,ell34,ell35,ell36
count,4.262000e+03,4262.0,4.262000e+03,4262.0,4262.0,4262.0,4262.0,4.262000e+03,4.262000e+03,4.262000e+03,...,4.262000e+03,4.262000e+03,4.262000e+03,4.262000e+03,4.262000e+03,4.262000e+03,4.262000e+03,4.262000e+03,4.262000e+03,4.262000e+03
mean,7.200000e-01,0.3,7.000000e-01,-1.0,0.0,1.0,3.5,2.286706e-09,2.078589e-09,1.903043e-09,...,2.288055e-10,2.045249e-10,1.811485e-10,1.613217e-10,1.431800e-10,1.267222e-10,1.112968e-10,9.773078e-11,8.577638e-11,7.437345e-11
std,1.110353e-16,0.0,2.220707e-16,0.0,0.0,0.0,0.0,9.030249e-10,1.143143e-09,8.769080e-10,...,3.194115e-11,2.712983e-11,2.313461e-11,2.027922e-11,1.713731e-11,1.482888e-11,1.239926e-11,1.091238e-11,8.921683e-12,7.540407e-12
min,7.200000e-01,0.3,7.000000e-01,-1.0,0.0,1.0,3.5,5.075263e-10,1.225567e-10,3.038688e-10,...,1.319937e-10,1.161248e-10,1.169752e-10,9.960836e-11,9.480818e-11,8.054565e-11,7.316575e-11,5.997823e-11,5.819787e-11,5.087779e-11
25%,7.200000e-01,0.3,7.000000e-01,-1.0,0.0,1.0,3.5,1.659332e-09,1.257986e-09,1.266880e-09,...,2.062474e-10,1.854765e-10,1.648687e-10,1.471571e-10,1.313811e-10,1.163175e-10,1.025522e-10,8.991890e-11,7.943382e-11,6.907277e-11
50%,7.200000e-01,0.3,7.000000e-01,-1.0,0.0,1.0,3.5,2.137508e-09,1.860261e-09,1.757861e-09,...,2.262848e-10,2.030887e-10,1.799418e-10,1.601002e-10,1.423961e-10,1.259744e-10,1.105838e-10,9.734929e-11,8.545252e-11,7.398224e-11
75%,7.200000e-01,0.3,7.000000e-01,-1.0,0.0,1.0,3.5,2.744331e-09,2.627518e-09,2.375482e-09,...,2.493000e-10,2.221095e-10,1.962156e-10,1.738807e-10,1.540260e-10,1.361707e-10,1.193818e-10,1.049980e-10,9.161083e-11,7.932900e-11
max,7.200000e-01,0.3,7.000000e-01,-1.0,0.0,1.0,3.5,8.186498e-09,1.081884e-08,7.602001e-09,...,3.630112e-10,3.058062e-10,2.800802e-10,2.428776e-10,2.127101e-10,1.968145e-10,1.780451e-10,1.404588e-10,1.179031e-10,1.027043e-10


In [13]:
feature_cov = np.cov(data[ell_bins].T)

In [14]:
emulator = Emulator.from_features(features=data[ell_bins].to_numpy()[:100])

In [15]:
emulator.train()

In [16]:
# posterior = emulator.sample_posterior(data[ell_bins])

In [17]:
gmm = GMM()
gmm.fit(data[ell_bins])

GaussianMixture()

In [57]:
nwalkers = 1000
ndim = len(ell_bins)
def log_prob(x):
    return  np.log(np.maximum(gmm.score(x.reshape(1, -1)), 1e-16 ))
p0 = np.mean(data[ell_bins].to_numpy(), axis=0)[np.newaxis, :]*(1 + np.random.normal(0, 1, size=(nwalkers, ndim))) #+ np.random.normal(0, 1, size=(nwalkers, ndim))

# with Pool(processes=4) as pool:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, threads=10)
out = sampler.run_mcmc(p0, 1000)

In [42]:
gmm.score(out[0][0].reshape((1, -1)))

19.532922662927433

In [56]:
help(sampler.sample)

Help on method sample in module emcee.ensemble:

sample(p0, lnprob0=None, rstate0=None, blobs0=None, iterations=1, thin=1, storechain=True, mh_proposal=None) method of emcee.ensemble.EnsembleSampler instance
    Advance the chain ``iterations`` steps as a generator.
    
    :param p0:
        A list of the initial positions of the walkers in the
        parameter space. It should have the shape ``(nwalkers, dim)``.
    
    :param lnprob0: (optional)
        The list of log posterior probabilities for the walkers at
        positions given by ``p0``. If ``lnprob is None``, the initial
        values are calculated. It should have the shape ``(k, dim)``.
    
    :param rstate0: (optional)
        The state of the random number generator.
        See the :attr:`Sampler.random_state` property for details.
    
    :param iterations: (optional)
        The number of steps to run.
    
    :param thin: (optional)
        If you only want to store and yield every ``thin`` samples in the
  

In [43]:
out

(array([[ 0.00202189, -0.00228694,  0.00908325, ...,  0.00036763,
          0.00019458, -0.00056194],
        [ 0.0018298 , -0.00195061,  0.00820604, ...,  0.0003238 ,
          0.00018257, -0.00051219],
        [ 0.00215876, -0.00221947,  0.00916308, ...,  0.00038466,
          0.00019295, -0.00057489],
        ...,
        [-0.00190803,  0.00196902, -0.00852156, ..., -0.00034839,
         -0.00017936,  0.00053088],
        [-0.00190885,  0.00166134, -0.00774936, ..., -0.00031654,
         -0.00017011,  0.00048248],
        [-0.00225032,  0.00235057, -0.00942221, ..., -0.00038479,
         -0.00019976,  0.00059233]]),
 array([2.97210138, 3.98306456, 2.66707088, 4.27943728, 4.76009618,
        1.54743392, 3.4088984 , 4.36168018, 3.68988299, 2.94953863,
        3.67183197, 2.53107148, 2.60481494, 3.60187904, 3.44112161,
        1.89058705, 2.87480111, 3.73211896, 3.2845173 , 3.34066528,
        3.71189443, 4.17005683, 4.1631577 , 4.26136459, 3.00252223,
        2.82969147, 3.11225488, 3